In [39]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Đọc file users.dat
users = pd.read_csv('dataset/users.dat', sep="::", engine='python', 
                    names=["UserID", "Gender", "Age", "Occupation", "Zip-code"])

# Đọc file ratings.dat
ratings = pd.read_csv('dataset/ratings.dat', sep="::", engine='python', 
                      names=["UserID", "MovieID", "Rating", "Timestamp"])

# Đọc file movies.dat
movies = pd.read_csv('dataset/movies.dat', sep="::", engine='python', 
                     names=["MovieID", "Title", "Genres"], encoding="ISO-8859-1")

# Hiển thị 5 dòng đầu tiên của mỗi DataFrame để kiểm tra
print("Users DataFrame:")
print(users.head())
print("\nRatings DataFrame:")
print(ratings.head())
print("\nMovies DataFrame:")
print(movies.head())



Users DataFrame:
   UserID Gender  Age  Occupation Zip-code
0       1      F    1          10    48067
1       2      M   56          16    70072
2       3      M   25          15    55117
3       4      M   45           7    02460
4       5      M   25          20    55455

Ratings DataFrame:
   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291

Movies DataFrame:
   MovieID                               Title                        Genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Com

In [40]:
# Tạo ma trận One-Hot cho các thể loại phim
genre_dummies = movies['Genres'].str.get_dummies(sep='|')

# Ghi nhãn cho các genre để sử dụng dễ dàng hơn
genre_dummies.columns = [genre.strip() for genre in genre_dummies.columns]

# Thêm MovieID làm chỉ mục cho genre_dummies
genre_dummies.index = movies['MovieID']

# Hiển thị ma trận One-Hot
print("\nOne-Hot Encoded Genres with MovieID as index:")
print(genre_dummies.head())



One-Hot Encoded Genres with MovieID as index:
         Action  Adventure  Animation  Children's  Comedy  Crime  Documentary  \
MovieID                                                                         
1             0          0          1           1       1      0            0   
2             0          1          0           1       0      0            0   
3             0          0          0           0       1      0            0   
4             0          0          0           0       1      0            0   
5             0          0          0           0       1      0            0   

         Drama  Fantasy  Film-Noir  Horror  Musical  Mystery  Romance  Sci-Fi  \
MovieID                                                                         
1            0        0          0       0        0        0        0       0   
2            0        1          0       0        0        0        0       0   
3            0        0          0       0        0        0 

In [41]:
user_id = 1  # Thay đổi user_id theo nhu cầu
user_ratings = ratings[ratings['UserID'] == user_id].pivot(index='UserID', columns='MovieID', values='Rating').fillna(0)

# Hiển thị ma trận ratings chỉ của user_id
print(f"\nUser Ratings for User ID {user_id}:")
print(user_ratings)


User Ratings for User ID 1:
MovieID  1     48    150   260   527   531   588   594   595   608   ...  \
UserID                                                               ...   
1           5     5     5     4     5     4     4     4     5     4  ...   

MovieID  2692  2762  2791  2797  2804  2918  3105  3114  3186  3408  
UserID                                                               
1           4     4     4     4     5     4     5     4     4     4  

[1 rows x 53 columns]


In [42]:
# Bước 6: Nhân giá trị đánh giá với ma trận thể loại
user_genre_scores = pd.DataFrame(0, index=[user_id], columns=genre_dummies.columns)

for movie_id, rating in user_ratings.iloc[0].items():
    if rating > 0:  # Chỉ xét những phim đã được đánh giá
        user_genre_scores += rating * genre_dummies.loc[movie_id]

# Hiển thị kết quả
print("\nUser Genre Scores:")
print(user_genre_scores)

# Bước 7: Chuẩn hóa điểm thể loại
# Chuyển đổi tổng điểm thành mảng NumPy trước khi chia
total_genre_scores = user_genre_scores.sum(axis=1).values.reshape(-1, 1)
user_genre_scores_normalized = user_genre_scores / total_genre_scores

# Hiển thị điểm thể loại đã được chuẩn hóa
print("\nNormalized User Genre Scores:")
print(user_genre_scores_normalized)



User Genre Scores:
   Action  Adventure  Animation  Children's  Comedy  Crime  Documentary  \
1      21         20         74          85      58      8            0   

   Drama  Fantasy  Film-Noir  Horror  Musical  Mystery  Romance  Sci-Fi  \
1     93       12          0       0       60        0       22      13   

   Thriller  War  Western  
1        11   10        0  

Normalized User Genre Scores:
     Action  Adventure  Animation  Children's    Comedy     Crime  \
1  0.043121   0.041068   0.151951    0.174538  0.119097  0.016427   

   Documentary     Drama   Fantasy  Film-Noir  Horror   Musical  Mystery  \
1          0.0  0.190965  0.024641        0.0     0.0  0.123203      0.0   

    Romance    Sci-Fi  Thriller       War  Western  
1  0.045175  0.026694  0.022587  0.020534      0.0  


In [43]:
# Bước 1: Lấy ID của các phim chưa được đánh giá
rated_movie_ids = user_ratings.columns[user_ratings.iloc[0] > 0].tolist()  # Lấy danh sách ID phim đã đánh giá
unrated_movie_ids = genre_dummies.index[~genre_dummies.index.isin(rated_movie_ids)]  # ID phim chưa đánh giá

# Bước 2: Lấy các phim chưa được đánh giá từ genre_dummies
unrated_genre_dummies = genre_dummies.loc[unrated_movie_ids]

# Bước 3: Nhân user_genre_scores_normalized với unrated_genre_dummies
# Chúng ta sẽ nhân từng cột của unrated_genre_dummies với user_genre_scores_normalized
suggested_scores = unrated_genre_dummies * user_genre_scores_normalized.values

# Hiển thị kết quả
print("\nSuggested Scores for Unrated Movies:")
print(suggested_scores)



Suggested Scores for Unrated Movies:
           Action  Adventure  Animation  Children's    Comedy     Crime  \
MovieID                                                                   
2        0.000000   0.041068        0.0    0.174538  0.000000  0.000000   
3        0.000000   0.000000        0.0    0.000000  0.119097  0.000000   
4        0.000000   0.000000        0.0    0.000000  0.119097  0.000000   
5        0.000000   0.000000        0.0    0.000000  0.119097  0.000000   
6        0.043121   0.000000        0.0    0.000000  0.000000  0.016427   
...           ...        ...        ...         ...       ...       ...   
3948     0.000000   0.000000        0.0    0.000000  0.119097  0.000000   
3949     0.000000   0.000000        0.0    0.000000  0.000000  0.000000   
3950     0.000000   0.000000        0.0    0.000000  0.000000  0.000000   
3951     0.000000   0.000000        0.0    0.000000  0.000000  0.000000   
3952     0.000000   0.000000        0.0    0.000000  0.000000 

In [44]:
# Bước 1: Tính tổng điểm gợi ý cho mỗi phim chưa được đánh giá
suggested_scores = unrated_genre_dummies * user_genre_scores_normalized.values

# Bước 2: Cộng các giá trị lại cho mỗi MovieID
suggested_scores_sum = suggested_scores.sum(axis=1)

# Bước 3: Tạo DataFrame để lưu điểm gợi ý cho mỗi bộ phim
suggested_movies = pd.DataFrame({
    'MovieID': suggested_scores_sum.index,
    'SuggestedScore': suggested_scores_sum.values
})

# Sắp xếp điểm gợi ý từ lớn đến bé
suggested_movies_sorted = suggested_movies.sort_values(by='SuggestedScore', ascending=False)

# Hiển thị kết quả đã sắp xếp
print("\nSuggested Movies Sorted by Score (Descending):")
print(suggested_movies_sorted)
# Giới hạn điểm gợi ý trong khoảng từ 0 đến 5
suggested_movies_sorted['SuggestedScore'] = suggested_movies_sorted['SuggestedScore'].clip(lower=0, upper=5)

print(suggested_movies_sorted)



Suggested Movies Sorted by Score (Descending):
      MovieID  SuggestedScore
1975     2081        0.613963
1974     2080        0.613963
1996     2102        0.568789
1972     2078        0.568789
2032     2138        0.542094
...       ...             ...
3520     3642        0.000000
1866     1970        0.000000
1865     1969        0.000000
396       404        0.000000
2427     2538        0.000000

[3830 rows x 2 columns]
      MovieID  SuggestedScore
1975     2081        0.613963
1974     2080        0.613963
1996     2102        0.568789
1972     2078        0.568789
2032     2138        0.542094
...       ...             ...
3520     3642        0.000000
1866     1970        0.000000
1865     1969        0.000000
396       404        0.000000
2427     2538        0.000000

[3830 rows x 2 columns]


In [45]:
import pandas as pd

# Đọc file users.dat
users = pd.read_csv('dataset/users.dat', sep="::", engine='python', 
                    names=["UserID", "Gender", "Age", "Occupation", "Zip-code"])

# Đọc file ratings.dat
ratings = pd.read_csv('dataset/ratings.dat', sep="::", engine='python', 
                      names=["UserID", "MovieID", "Rating", "Timestamp"])

# Đọc file movies.dat
movies = pd.read_csv('dataset/movies.dat', sep="::", engine='python', 
                     names=["MovieID", "Title", "Genres"], encoding="ISO-8859-1")


# Hiển thị 5 dòng đầu tiên của mỗi DataFrame để kiểm tra
print("Users DataFrame:")
print(users.head())
print("\nRatings DataFrame:")
print(ratings.head())
print("\nMovies DataFrame:")
print(movies.head())


Users DataFrame:
   UserID Gender  Age  Occupation Zip-code
0       1      F    1          10    48067
1       2      M   56          16    70072
2       3      M   25          15    55117
3       4      M   45           7    02460
4       5      M   25          20    55455

Ratings DataFrame:
   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291

Movies DataFrame:
   MovieID                               Title                        Genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Com

In [46]:
from sklearn.model_selection import train_test_split

# Chia dữ liệu ratings thành hai tập train và test
train, test = train_test_split(ratings, test_size=0.2, random_state=42)

# Kiểm tra kích thước của mỗi tập
print("Kích thước tập train:", train.shape)
print("Kích thước tập test:", test.shape)


Kích thước tập train: (800167, 4)
Kích thước tập test: (200042, 4)


In [47]:
ratings = train

In [48]:
import pandas as pd
import re

movies = movies.sample(n=3000, random_state=2)

ratings = ratings.merge(movies[['MovieID']], on='MovieID', how='inner')

print(movies)
print(ratings)

      MovieID                                         Title  \
2586     2655  Howling II: Your Sister Is a Werewolf (1985)   
3032     3101                       Fatal Attraction (1987)   
3273     3342                                  Birdy (1984)   
1866     1935                How Green Was My Valley (1941)   
3138     3207              Snows of Kilimanjaro, The (1952)   
...       ...                                           ...   
2097     2166                     Return to Paradise (1998)   
2070     2139                    Secret of NIMH, The (1982)   
1562     1603                                  Mimic (1997)   
3337     3406             Captain Horatio Hornblower (1951)   
881       893                           Mother Night (1996)   

                    Genres  
2586                Horror  
3032              Thriller  
3273             Drama|War  
1866                 Drama  
3138             Adventure  
...                    ...  
2097         Drama|Romance  
2070  Anima

In [49]:
# from sklearn.model_selection import train_test_split

# def split_user_group(group, test_size=0.2):
#     if len(group) < 2:
#         return group, pd.DataFrame()
#     train, test = train_test_split(group, test_size=test_size, random_state=1)
#     return train, test

# train_list = []
# test_list = []

# for user_id, group in filtered_books_rating_df.groupby('User_id'):
#     train_group, test_group = split_user_group(group)
#     train_list.append(train_group)
#     test_list.append(test_group)

# train_data = pd.concat(train_list)
# test_data = pd.concat(test_list)

# print("Tập train:")
# print(train_data)
# print("Tập test:")
# print(test_data)

In [50]:
# books_rating_df = train_data

In [51]:
# import pandas as pd

# def count_books_per_user(data):
#     user_book_counts = data.groupby('User_id')['book_id'].count().reset_index()
#     user_book_counts.columns = ['User_id', 'Rated_Books_Count']
#     return user_book_counts

# train_user_counts = count_books_per_user(books_rating_df)

# train_user_counts = train_user_counts.sort_values(by='Rated_Books_Count', ascending=False)

# print("Số lượng sách đã đánh giá trong train_data cho từng User_id (sắp xếp giảm dần):")
# print(train_user_counts)


In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score
import pandas as pd
import numpy as np

# tfidf_vectorizer = TfidfVectorizer(stop_words='english')
# tfidf_matrix = tfidf_vectorizer.fit_transform(books['description'])
# cosine_sim_description = cosine_similarity(tfidf_matrix)
# cosine_sim_description_df = pd.DataFrame(cosine_sim_description, index=books['book_id'], columns=books['book_id'])
# print(cosine_sim_description_df)


In [53]:
# mlb_categories = MultiLabelBinarizer()
# mlb_authors = MultiLabelBinarizer()
# categories_encoded = mlb_categories.fit_transform(books['categories'])
# authors_encoded = mlb_authors.fit_transform(books['authors'])


In [54]:
movies['Genres'] = movies['Genres'].apply(lambda x: x.split('|'))


mlb_genres = MultiLabelBinarizer()
genres_encoded = mlb_genres.fit_transform(movies['Genres'])


In [55]:
# def check_all_zero(encoded_matrix, feature_name, df):
#     all_zero_rows = []
#     for idx, row in enumerate(encoded_matrix):
#         if np.all(row == 0):
#             all_zero_rows.append(idx)
    
#     if all_zero_rows:
#         print(f"Các hàng toàn 0 trong {feature_name}: {all_zero_rows}")
#         zero_id_list = df.iloc[all_zero_rows]['book_id'].tolist()  # Giả sử cột ID có tên là 'id'
#         print(f"Các ID của sách có hàng mã hóa toàn 0 trong {feature_name}: {zero_id_list}")
#     else:
#         print(f"Không có hàng nào toàn 0 trong {feature_name}.")

# check_all_zero(categories_encoded, "categories", books)
# check_all_zero(authors_encoded, "authors", books)

Khoảng cách L2 (Euclidean Distance)

In [56]:
import pandas as pd
import numpy as np
import faiss

# Hàm tính toán ma trận tương đồng bằng FAISS
def faiss_similarity(encoded_matrix):
    encoded_matrix = encoded_matrix.astype(np.float32)
    index = faiss.IndexFlatL2(encoded_matrix.shape[1])
    index.add(encoded_matrix)
    distances, _ = index.search(encoded_matrix, k=encoded_matrix.shape[0])
    return 1 - (distances / np.max(distances))

# # Tính toán ma trận tương đồng cho thể loại
# similarity_categories = faiss_similarity(categories_encoded)

# # Tính toán ma trận tương đồng cho tác giả
# similarity_authors = faiss_similarity(authors_encoded)

# # Chuyển thành DataFrame
# similarity_categories_df = pd.DataFrame(similarity_categories, index=books['book_id'], columns=books['book_id'])
# similarity_authors_df = pd.DataFrame(similarity_authors, index=books['book_id'], columns=books['book_id'])


# Tính toán ma trận tương đồng cho thể loại
similarity_genres = faiss_similarity(genres_encoded)

similarity_genres_df = pd.DataFrame(similarity_genres, index=movies['MovieID'], columns=movies['MovieID'])


# Xuất kết quả
print("Ma trận tương đồng cho thể loại:")
print(similarity_genres_df)



Ma trận tương đồng cho thể loại:
MovieID  2655  3101  3342  1935  3207  509   3875  726   3545  1375  ...  \
MovieID                                                              ...   
2655      1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
3101      1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
3342      1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
1935      1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
3207      1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
2166      1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
2139      1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
1603      1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
3406      1.0   1.0   0.9   0.9   0.9   0.9   0.9   0.9   0.9   0.9  ...   
893       1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0

In [57]:
alpha = 0.2  # (description)
beta = 0.4  # (categories)
gamma = 0.4  # (authors)

# similarity_matrix = (
#     alpha * cosine_sim_description_df +
#     beta * similarity_categories_df +
#     gamma * similarity_authors_df
# ) 

similarity_matrix = similarity_genres_df

print(similarity_matrix)

MovieID  2655  3101  3342  1935  3207  509   3875  726   3545  1375  ...  \
MovieID                                                              ...   
2655      1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
3101      1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
3342      1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
1935      1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
3207      1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
2166      1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
2139      1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
1603      1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   
3406      1.0   1.0   0.9   0.9   0.9   0.9   0.9   0.9   0.9   0.9  ...   
893       1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   

MovieID  24

In [58]:
all_book_ids = movies['MovieID'].tolist()
print(all_book_ids)

[2655, 3101, 3342, 1935, 3207, 509, 3875, 726, 3545, 1375, 1970, 2688, 738, 282, 3272, 2793, 981, 1177, 1155, 2691, 1248, 1158, 3393, 3896, 3550, 2532, 1226, 3776, 345, 1693, 3817, 1127, 1342, 759, 1734, 2111, 3159, 229, 1977, 1176, 892, 744, 1098, 1999, 2480, 3114, 340, 142, 2298, 2088, 1038, 1097, 940, 1194, 3512, 842, 304, 765, 551, 2671, 575, 381, 800, 3752, 524, 1284, 1303, 1932, 2903, 1860, 2153, 2252, 3747, 1180, 1011, 3676, 3866, 3210, 103, 942, 422, 367, 131, 3009, 2435, 1324, 3706, 133, 2651, 173, 1926, 2487, 2603, 1948, 1738, 2197, 877, 1160, 2535, 997, 1489, 1594, 123, 1965, 970, 2107, 899, 73, 2518, 894, 3333, 283, 1585, 3035, 3703, 984, 1680, 993, 1152, 1041, 868, 1772, 1149, 2471, 2410, 3659, 590, 531, 3793, 1113, 1719, 19, 1984, 3062, 3320, 2044, 120, 1423, 1922, 1464, 1687, 1956, 2380, 93, 3910, 2694, 2121, 1834, 2742, 3599, 3319, 2612, 2812, 3415, 3800, 1320, 2464, 2071, 1919, 2533, 3546, 1963, 3020, 2573, 3539, 3384, 1718, 2260, 2396, 3179, 341, 1095, 1050, 239, 1602

In [ ]:
import pandas as pd

def check_index_matches(df, book_ids):
    # Chuyển chỉ mục của DataFrame thành một tập hợp
    df_index_set = set(df.index)
    # Chuyển book_ids thành một tập hợp
    book_ids_set = set(book_ids)
    
    # Kiểm tra xem hai tập hợp có bằng nhau không
    if df_index_set == book_ids_set:
        return True
    else:
        return False

result = check_index_matches(similarity_matrix, all_book_ids)
print("Chỉ mục của df trùng khớp với book_ids:", result) 


Chỉ mục của df trùng khớp với book_ids: True


In [60]:
# import pandas as pd

# similar_books_list = []

# for book_id_1 in cosine_sim_matrix.index:
#     for book_id_2 in cosine_sim_matrix.columns:
#         if book_id_1 != book_id_2:
#             similarity = cosine_sim_matrix.loc[book_id_1, book_id_2]
#             if 0 < similarity < 0.9:
#                 similar_books_list.append({
#                     'book_id_1': book_id_1,
#                     'book_id_2': book_id_2,
#                     'similarity_score': similarity
#                 })

# similar_books_df = pd.DataFrame(similar_books_list)

# top_similar_books = similar_books_df.sort_values(by='similarity_score', ascending=False).head(5)

# if not top_similar_books.empty:
#     print("Các cặp sách có giá trị tương đồng cosine > 0 và < 0.9:")
#     for index, row in top_similar_books.iterrows():
#         book_id_1 = row['book_id_1']
#         book_id_2 = row['book_id_2']
#         similarity = row['similarity_score']

#         title1 = books_data_df[books_data_df['book_id'] == book_id_1]['Title'].values[0]
#         title2 = books_data_df[books_data_df['book_id'] == book_id_2]['Title'].values[0]
#         categories1 = books_data_df[books_data_df['book_id'] == book_id_1]['categories'].values[0]
#         categories2 = books_data_df[books_data_df['book_id'] == book_id_2]['categories'].values[0]

#         print(f"Sách '{title1}' với thể loại {categories1} và Sách '{title2}' với thể loại {categories2} có giá trị tương đồng: {similarity}")
# else:
#     print("Không có cặp sách nào có giá trị tương đồng trong khoảng (0, 0.9).")


In [61]:
# import pandas as pd

# book_title = "Brave New World Revisited"
# book_id = books_data_df.loc[books_data_df['Title'] == book_title, 'book_id'].values
# print(book_id)
# if len(book_id) > 0:
#     # Tìm user_id đã đánh giá sách này
#     user_ids = books_rating_df.loc[books_rating_df['book_id'].isin(book_id), 'User_id'].unique()
#     print("User IDs đã đánh giá sách 'History of Magic and the Occult':", user_ids)
# else:
#     print("Không tìm thấy sách có tiêu đề 'History of Magic and the Occult'.")


In [62]:
# book_id = 24

# book_info = books_data_df.loc[books_data_df['book_id'] == book_id]

# if not book_info.empty:
#     title = book_info['Title'].values[0]
#     description = book_info['description'].values[0]
#     categories = book_info['categories'].values[0]
#     author = book_info['authors'].values[0]
#     print(f"Thông tin cuốn sách với book_id {book_id}:")
#     print(f"Tiêu đề: '{title}'")
#     print(f"Mô tả: '{description}'")
#     print(f"Thể loại: {categories}")
#     print(f"Tác giả: {author}")
# else:
#     print(f"Không tìm thấy sách với book_id {book_id}.")


In [63]:
# book_id = 64109

# book_info = books_data_df.loc[books_data_df['book_id'] == book_id]

# if not book_info.empty:
#     title = book_info['Title'].values[0]
#     description = book_info['description'].values[0]
#     categories = book_info['categories'].values[0]
#     author = book_info['authors'].values[0]
#     print(f"Thông tin cuốn sách với book_id {book_id}:")
#     print(f"Tiêu đề: '{title}'")
#     print(f"Mô tả: '{description}'")
#     print(f"Thể loại: {categories}")
#     print(f"Tác giả: {author}")
# else:
#     print(f"Không tìm thấy sách với book_id {book_id}.")


In [64]:
user_rating_matrix = ratings.pivot_table(index="UserID", columns="MovieID", values="Rating")

In [ ]:
def calculate_similarity(user_id, movie_id, rated_movies):
    if movie_id not in similarity_matrix.index:
        return None

    similarities = similarity_matrix.loc[movie_id]  

    ratings_for_rated_movies = user_rating_matrix.loc[user_id]

    if ratings_for_rated_movies.empty:
        return None

    weighted_sum = 0
    similarity_sum = 0
    for rated_movie in rated_movies:
        weighted_sum = weighted_sum + (similarities.loc[rated_movie]*ratings_for_rated_movies[rated_movie])
        similarity_sum += np.abs(similarities.loc[rated_movie])
   

    if similarity_sum > 0:
        average_similarity = weighted_sum / similarity_sum
        return average_similarity
    
    return None

In [66]:
def recommend_books(user_id, top_k=10):
    rated_books = ratings[ratings['UserID'] == user_id]['MovieID'].unique()
    # rated_books = [book_id for book_id in rated_books if book_id in similarity_matrix.index]

    if len(rated_books) == 0:
        print(f"Người dùng {user_id} chưa đánh giá sách nào.")
        return []

    similarity_scores = {}

    # test_books = test_data['book_id'].unique()
    # unrated_books = books_data_df['book_id'][~books_data_df['book_id'].isin(rated_books)]
    # unrated_books = unrated_books[unrated_books.isin(test_books)]


    # for book_id in unrated_books:
    #     similarity_score = calculate_similarity(book_id, rated_books)
        
    #     if similarity_score is not None:
    #         similarity_scores[book_id] = similarity_score

    for book_id in all_book_ids:
        if(book_id not in rated_books):
            similarity_score = calculate_similarity(user_id, book_id, rated_books)

            if similarity_score is not None:
                similarity_scores[book_id] = similarity_score

    if similarity_scores:
        suggested_books = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]

        return [[book[0], similarity_scores[book[0]]] for book in suggested_books]
    else:
        print(f"Không có sách nào để gợi ý cho người dùng {user_id}.")
        return []



In [ ]:
user_id = 87
top_k = 10

suggested_books = recommend_books(user_id, top_k)

if suggested_books:
    for book_id, similarity_score in suggested_books:
        title_series = movies.loc[movies['MovieID'] == book_id, 'Title']
        categories_series = movies.loc[movies['MovieID'] == book_id, 'Genres']
        if not title_series.empty and not categories_series.empty:
            title = title_series.values[0]
            categories = categories_series.values[0]
            print(f"Sách '{title}' (ID: {book_id}) với thể loại {categories}, Điểm tương đồng: {similarity_score:.4f}")
        else:
            print(f"Không tìm thấy sách với ID: {book_id}") 
else:
    print("Không có cuốn sách nào để gợi ý.")


Sách 'Tough and Deadly (1995)' (ID: 591) với thể loại ['Action', 'Drama', 'Thriller'], Điểm tương đồng: 3.0037
Sách 'Condition Red (1995)' (ID: 624) với thể loại ['Action', 'Drama', 'Thriller'], Điểm tương đồng: 3.0037
Sách 'Fire Down Below (1997)' (ID: 1626) với thể loại ['Action', 'Drama', 'Thriller'], Điểm tương đồng: 3.0037
Sách 'Smilla's Sense of Snow (1997)' (ID: 1480) với thể loại ['Action', 'Drama', 'Thriller'], Điểm tương đồng: 3.0037
Sách 'Born American (1986)' (ID: 3443) với thể loại ['Action', 'Drama', 'Thriller'], Điểm tương đồng: 3.0037
Sách 'Outbreak (1995)' (ID: 292) với thể loại ['Action', 'Drama', 'Thriller'], Điểm tương đồng: 3.0037
Sách 'Guardian Angel (1994)' (ID: 51) với thể loại ['Action', 'Drama', 'Thriller'], Điểm tương đồng: 3.0037
Sách 'Rocky III (1982)' (ID: 2410) với thể loại ['Action', 'Drama'], Điểm tương đồng: 2.9967
Sách 'Falling Down (1993)' (ID: 3020) với thể loại ['Action', 'Drama'], Điểm tương đồng: 2.9967
Sách 'Target (1995)' (ID: 139) với thể loại

In [ ]:
import pandas as pd
from collections import Counter

def get_interested_genres(user_id, ratings_df, movies_df, min_rating=2):
    user_ratings = ratings_df[(ratings_df['UserID'] == user_id) & (ratings_df['Rating'] >= min_rating)]
    
    user_movies = pd.merge(user_ratings, movies_df, on='MovieID')
    
    user_movies['Genres'] = user_movies['Genres'].fillna('') 
    user_movies['Genres'] = user_movies['Genres'].astype(str)  
    
    genres = user_movies['Genres'].str.split('|').sum() 
    
    genre_counts = Counter(genres)
    
    interested_genres = genre_counts.most_common()
    return interested_genres

user_id = 87 
interested_genres = get_interested_genres(user_id, ratings, movies)

print(f"Các thể loại mà người dùng {user_id} hứng thú:")
for genre, count in interested_genres:
    print(f"{genre}: {count}")


Các thể loại mà người dùng 87 hứng thú:
['Action', 'Thriller']: 4
['Action']: 2
['Action', 'Drama']: 2
['Action', 'Drama', 'War']: 2
['Action', 'Western']: 1
['Action', 'Crime', 'Drama']: 1
['Action', 'Adventure', 'Thriller']: 1
['Action', 'Adventure', 'Sci-Fi']: 1
['Action', 'Adventure', 'Comedy', 'Romance']: 1
['Action', 'Sci-Fi', 'Thriller']: 1
['Action', 'Adventure', 'Fantasy', 'Sci-Fi']: 1
['Drama']: 1
['Action', 'Adventure', 'Drama', 'Sci-Fi', 'War']: 1
['Action', 'Adventure']: 1
['Action', 'Comedy', 'Crime', 'Drama']: 1
['Action', 'Sci-Fi']: 1
['Action', 'Drama', 'Romance']: 1


In [ ]:
user_ids = test['UserID'].unique().tolist()

print("Danh sách UserID độc nhất:")
print(user_ids)


Danh sách UserID độc nhất:
[5412, 5440, 368, 425, 4942, 4668, 2907, 2376, 3824, 5605, 4414, 5709, 5812, 285, 5491, 824, 349, 817, 3228, 4626, 5805, 4732, 1800, 3163, 1422, 5172, 2892, 5166, 58, 411, 3272, 2421, 1975, 1966, 5785, 3415, 1788, 4052, 4887, 3878, 2817, 5809, 288, 927, 4509, 1460, 4823, 1259, 5371, 1202, 1005, 5519, 2857, 3936, 4588, 2947, 3594, 5447, 4868, 4384, 3189, 4177, 5755, 3807, 5996, 889, 3919, 3605, 1958, 709, 869, 1121, 5458, 5702, 672, 2665, 1179, 3822, 4388, 982, 99, 3985, 1391, 1540, 6002, 3995, 1285, 1339, 2344, 832, 3044, 371, 4415, 3389, 5795, 2445, 5213, 2986, 4387, 3180, 5433, 3592, 3144, 5227, 5102, 1317, 738, 2919, 3359, 5643, 3976, 4592, 579, 3650, 215, 4139, 1613, 5039, 536, 4700, 3391, 678, 5891, 3503, 1015, 5560, 4007, 4416, 10, 5463, 5763, 4373, 4408, 4815, 5501, 5283, 5406, 18, 3841, 1051, 5089, 1254, 799, 3346, 3957, 1172, 4064, 2996, 1062, 667, 424, 1926, 1951, 854, 4169, 1919, 4506, 412, 2771, 170, 1184, 4066, 2348, 1390, 216, 2380, 5646, 752, 1

In [ ]:
import random

user_ids_sample = random.sample(user_ids, 500)
user_ids = user_ids_sample
print("Danh sách 1000 UserID ngẫu nhiên:")
print(user_ids)


Danh sách 1000 UserID ngẫu nhiên:
[2531, 1959, 4930, 280, 1462, 3323, 678, 3215, 3644, 4411, 4299, 3994, 2606, 2006, 1407, 5240, 839, 4394, 2341, 1499, 5866, 2240, 4948, 5231, 429, 2561, 2793, 2057, 4003, 249, 405, 1020, 2929, 944, 3691, 3156, 4461, 2610, 1380, 3540, 1123, 5545, 4206, 2833, 2377, 2732, 3049, 6026, 820, 4170, 5990, 2626, 19, 4251, 3475, 772, 4711, 4161, 4475, 1598, 4348, 587, 5014, 3468, 3375, 4259, 840, 2665, 579, 2243, 2192, 817, 4911, 4116, 1296, 1213, 5633, 724, 3113, 3699, 1330, 1964, 269, 4019, 1455, 5653, 3954, 4778, 2139, 4154, 4150, 651, 4083, 2084, 3470, 1172, 3760, 397, 1357, 1452, 4756, 3355, 1190, 2753, 2404, 2532, 1470, 3452, 3066, 4607, 106, 4776, 4291, 1546, 2079, 1430, 3070, 357, 2569, 2711, 1711, 442, 1035, 4770, 4803, 44, 3245, 4639, 1042, 1265, 3706, 5911, 1645, 425, 1191, 4156, 1822, 5456, 2120, 5008, 960, 3564, 3650, 3948, 5160, 2296, 5297, 736, 696, 3345, 40, 1460, 1229, 3446, 4425, 2743, 1166, 3874, 2565, 3692, 1617, 3297, 1878, 1388, 4781, 2399,

In [ ]:
import numpy as np

total_tp = 0  # Tổng số True Positives
total_fp = 0  # Tổng số False Positives
total_fn = 0  # Tổng số False Negatives
total_squared_error = 0  # Tổng bình phương sai số
total_absolute_error = 0  # Tổng sai số tuyệt đối
total_predictions = 0  # Tổng số dự đoán được sử dụng cho MSE, RMSE, MAE


for user_id in user_ids:

    actual_books = test[test['UserID'] == user_id][['MovieID', 'Rating']].values.tolist()
    
    suggested_books = recommend_books(user_id, top_k=10) 
    
    actual_dict = dict(actual_books)
    
    suggested_book_ids = set([book_id for book_id, _ in suggested_books])
    actual_book_ids = set([book_id for book_id, _ in actual_books])

    # TP (True Positives), FP (False Positives) và FN (False Negatives) cho use hiện tại
    tp = len(suggested_book_ids.intersection(actual_book_ids))
    fp = len(suggested_book_ids - actual_book_ids)
    fn = len(actual_book_ids - suggested_book_ids)
    
    total_tp += tp
    total_fp += fp
    total_fn += fn

    # MSE, RMSE, MAE
    for book_id, predicted_score in suggested_books:
        if book_id in actual_dict:
            actual_score = actual_dict[book_id]
            total_squared_error += (actual_score - predicted_score) ** 2
            total_absolute_error += abs(actual_score - predicted_score)
            total_predictions += 1

precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

mse = total_squared_error / total_predictions if total_predictions > 0 else 0
rmse = np.sqrt(mse)
mae = total_absolute_error / total_predictions if total_predictions > 0 else 0

print("Overall Precision:", precision)
print("Overall Recall:", recall)
print("Overall F1-score:", f1_score)
print("Overall MSE:", mse) 
print("Overall RMSE:", rmse)
print("Overall MAE:", mae)


Overall Precision: 0.0182
Overall Recall: 0.005435107208982859
Overall F1-score: 0.00837051004921124
Overall MSE: 0.877522480008134
Overall RMSE: 0.9367616986235795
Overall MAE: 0.741880926062186
